In [1]:
import json
import re
import pandas as pd
from ranking import get_lemmas_len, get_hyponyms, get_frequencies
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

## Basicness ranking


In [2]:
with open('dataset_basic_advanced_TLN2023/1.json', 'r') as file:
    contenuto_json = json.load(file)

synset_target = [0 if answer == 'basic' else 1 for answer in contenuto_json['answers']]

synset_values = re.findall(r"Synset\(.*?\)", str(contenuto_json['dataset']))
synset_names = [synset_values[i].replace("Synset('", "").replace("')", "").split(".")[0] for i in range(0, len(synset_values))]

In [10]:
name_test = ['dog', 'rose', 'hello', 'serendipity', 'positivity', 'bye', 'girl']
lemma_test = get_lemmas_len(name_test)
hyp_test = get_hyponyms(name_test)
freq_test = get_frequencies(name_test)
y_test = [0, 0, 0, 1, 1, 0, 0]

df_test = pd.DataFrame({'Synset':name_test, 'Lunghezza lemma': lemma_test, 'Numero iponimi': hyp_test, 'Frequenza': freq_test})

df_test

,Synset,Lunghezza lemma,Numero iponimi,Frequenza
0,dog,3,24,42
1,rose,4,39,5
2,hello,5,0,1
3,serendipity,11,0,0
4,positivity,10,2,0
5,bye,3,0,0
6,girl,4,33,80


In [11]:
lemma_train = get_lemmas_len(synset_names)
hyp_train = get_hyponyms(synset_names)
freq_train = get_frequencies(synset_names)

df = pd.DataFrame({'Synset':synset_names, 'Lunghezza lemma': lemma_train, 'Numero iponimi': hyp_train, 'Frequenza': freq_train})

df

,Synset,Lunghezza lemma,Numero iponimi,Frequenza
0,war,3,14,78
1,fiefdom,7,0,0
2,bed,3,47,51
3,return_on_invested_capital,26,0,0
4,texture,7,5,13
...,...,...,...,...
499,reading,7,28,22
500,sanctimoniousness,17,0,0
501,chalcedony,10,7,0
502,stopcock,8,0,0


In [12]:
classifier = DecisionTreeClassifier(random_state=0)
classifier.fit(df[['Lunghezza lemma', 'Numero iponimi', 'Frequenza']], synset_target)
#cross_val_score(classifier, df[['Lunghezza lemma', 'Numero iponimi']], synset_target, cv=10)

DecisionTreeClassifier(random_state=0)

In [6]:
classifier.score(df[['Lunghezza lemma', 'Numero iponimi', 'Frequenza']], synset_target)

0.9166666666666666

In [13]:
y = classifier.predict(df_test[['Lunghezza lemma', 'Numero iponimi', 'Frequenza']])
y

array([0, 0, 1, 1, 1, 1, 0])

In [14]:
classifier.score(df_test[['Lunghezza lemma', 'Numero iponimi', 'Frequenza']], y_test)

0.7142857142857143